# 1.3 Prompt Engineering - Essential TechniquesThis notebook covers **core prompt engineering techniques** for AI applications:- **Chain-of-Thought (CoT)**: Encouraging step-by-step reasoning- **ReAct Pattern**: Combining Reasoning and Action- **Few-Shot Learning**: Learning from examples- **Delimiters & Structure**: Organizing prompts for clarity- **Prompt Chaining**: Breaking complex tasks into stepsThese patterns are fundamental to building effective AI applications.<a target="_blank" href="https://githubtocolab.com/IT-HUSET/ai-agenter-2025/blob/main/exercises/openai/1.4-prompt-engineering.ipynb">  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
%pip install openai~=2.1 python-dotenv~=1.0 --upgrade --quiet

In [ ]:
import os
from openai import OpenAI

# Check if running in Google Colab
try:
    from google.colab import userdata
    IN_COLAB = True
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ Running in Google Colab - API key loaded from secrets")
except ImportError:
    IN_COLAB = False
    try:
        from dotenv import load_dotenv, find_dotenv
        load_dotenv(find_dotenv())
        print("✅ Running locally - API key loaded from .env file")
    except ImportError:
        print("⚠️ python-dotenv not installed")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

if not os.getenv("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY not found!")
    if IN_COLAB:
        print("   → Click the key icon (🔑) in the left sidebar and add 'OPENAI_API_KEY'")
else:
    print("✅ Setup complete")

---

## Part 1: Chain-of-Thought (CoT) Prompting

**Problem**: LLMs sometimes jump to conclusions without showing their work.

**Solution**: Explicitly ask the model to "think step by step".

### Without CoT

In [ ]:
# Direct question - no reasoning shown
response = client.responses.create(
    model="gpt-5-mini",
    input="What is 47 × 23?"
)

print("Without CoT:")
print(response.output_text)

### With CoT

In [ ]:
# CoT prompt - shows reasoning
cot_prompt = """Calculate 47 × 23. Show your step-by-step reasoning:

Step 1: Break down the multiplication
Step 2: Calculate partial products
Step 3: Sum the results
Step 4: State the final answer
"""

response = client.responses.create(
    model="gpt-5-mini",
    input=cot_prompt
)

print("With CoT:")
print(response.output_text)

### CoT for Complex Problems

CoT is especially powerful for:
- Math problems
- Logic puzzles
- Multi-step reasoning
- Debugging code

In [ ]:
problem = """A train leaves Station A at 2:00 PM traveling at 60 mph. 
Another train leaves Station B (180 miles away) at 3:00 PM traveling at 90 mph toward Station A.
At what time will they meet?

Think through this step by step:
1. Calculate distance traveled by first train before second train starts
2. Calculate remaining distance between trains when both are moving
3. Calculate combined speed
4. Calculate time to meet
5. Determine the meeting time
"""

response = client.responses.create(
    model="gpt-5-mini",
    input=problem,
    temperature=0
)

print(response.output_text)

### 🎯 Exercise 1: Practice CoT

**Task:** Use CoT to solve this problem:

> "A farmer has chickens and rabbits. There are 35 heads and 94 legs total. How many chickens and how many rabbits?"

**Hint:** Structure your prompt to guide the model through:
1. Define variables
2. Set up equations
3. Solve the system
4. Verify the answer

In [ ]:
# YOUR CODE HERE
cot_problem = """TODO: Write your CoT prompt"""

# response = client.responses.create(
#     model="gpt-5-mini",
#     input=cot_problem,
#     temperature=0
# )
# print(response.output_text)

---

## Part 2: ReAct Pattern (Reason + Act)

**ReAct** combines reasoning with action. It's the foundation of most agent frameworks.

**Pattern:**
```
Thought: What do I need to do?
Action: call_tool(params)
Observation: Result from tool
Thought: What does this mean?
Answer: Final response
```

### Simple ReAct Example

Let's implement a basic ReAct loop manually (before using frameworks).

In [ ]:
# Define a simple tool
def get_weather(city: str) -> str:
    """Fake weather API"""
    weather_data = {
        "Stockholm": "5°C, cloudy, 80% chance of rain",
        "London": "8°C, foggy",
        "Paris": "10°C, sunny",
        "Tokyo": "15°C, clear"
    }
    return weather_data.get(city, "Weather data not available")

# Test it
print(get_weather("Stockholm"))

In [ ]:
# ReAct prompt that encourages the pattern
react_prompt = """You are a helpful assistant that can check the weather.

When answering questions, follow this pattern:
1. Thought: Analyze what you need to do
2. Action: Specify if you need to use the get_weather tool
3. Answer: Provide the final response

Available tool:
- get_weather(city: str) -> str: Returns weather for a city

Question: What's the weather like in Stockholm and should I bring an umbrella?

Let's think step by step:
"""

response = client.responses.create(
    model="gpt-5-mini",
    input=react_prompt,
    temperature=0
)

print(response.output_text)

### Manual ReAct Loop

Let's implement the loop manually to understand how it works.

In [ ]:
def manual_react_loop(question: str, max_iterations=3):
    """Manual implementation of ReAct pattern"""
    
    system_prompt = """You are a helpful assistant with access to tools.

Follow this pattern:
Thought: [your reasoning]
Action: [tool_name(parameters)] OR Answer: [final answer]
    
Available tools:
- get_weather(city: str): Get weather for a city

Always show your thought process. When you have enough information, provide a final Answer.
"""
    
    conversation = f"{system_prompt}\n\nQuestion: {question}\n\n"
    
    for iteration in range(max_iterations):
        print(f"\n--- Iteration {iteration + 1} ---\n")
        
        response = client.responses.create(
            model="gpt-5-mini",
            input=conversation,
            temperature=0
        )
        
        output = response.output_text
        print(output)
        
        # Check if we have a final answer
        if "Answer:" in output:
            print("\n✅ Agent reached final answer")
            return output
        
        # Extract action (simplified parsing)
        if "get_weather(" in output:
            # Parse the city (very simplified)
            import re
            match = re.search(r'get_weather\(["\']([^"\']+)["\']\)', output)
            if match:
                city = match.group(1)
                weather = get_weather(city)
                observation = f"\nObservation: {weather}\n"
                print(observation)
                conversation += output + observation
            else:
                print("\n❌ Could not parse action")
                break
        else:
            conversation += output
    
    return conversation

# Test it
result = manual_react_loop("Compare the weather in Stockholm and Paris. Which city is warmer?")

### 🎯 Exercise 2: Implement ReAct

**Task:** Create a ReAct agent that can use multiple tools:
- `get_temperature(city)`: Returns temperature only
- `get_population(city)`: Returns city population
- `calculate(expression)`: Evaluates math expressions

**Question:** "What's the average temperature between Stockholm and Tokyo, and what's their combined population?"

In [ ]:
# YOUR CODE HERE

# Define tools
def get_temperature(city: str) -> float:
    temps = {"Stockholm": 5, "Tokyo": 15, "Paris": 10, "London": 8}
    return temps.get(city, 0)

def get_population(city: str) -> int:
    pops = {"Stockholm": 975000, "Tokyo": 14000000, "Paris": 2161000, "London": 9000000}
    return pops.get(city, 0)

def calculate(expression: str) -> float:
    try:
        return eval(expression, {"__builtins__": {}}, {})
    except:
        return "Error"

# TODO: Implement a ReAct loop that can use these tools


---

## Part 3: Few-Shot Learning

**Few-shot learning**: Provide examples to guide the model's behavior.

**When to use:**
- Teaching new patterns or formats
- Establishing consistent style or tone
- Complex tasks that benefit from examples
- Domain-specific outputs

### Zero-Shot vs Few-Shot

In [ ]:
# Zero-shot: No examples
zero_shot = """Classify the sentiment of this review as Positive, Negative, or Neutral:

Review: "The product works okay, but the packaging was damaged."
"""

response = client.responses.create(
    model="gpt-5-mini",
    input=zero_shot,
    temperature=0
)

print("Zero-shot:")
print(response.output_text)

In [ ]:
# Few-shot: With examples
few_shot = """Classify the sentiment of reviews as Positive, Negative, or Neutral:

Review: "Absolutely amazing! Best purchase ever."
Sentiment: Positive

Review: "Terrible quality, broke after one day."
Sentiment: Negative

Review: "It's fine, nothing special."
Sentiment: Neutral

Review: "The product works okay, but the packaging was damaged."
Sentiment:
"""

response = client.responses.create(
    model="gpt-5-mini",
    input=few_shot,
    temperature=0
)

print("Few-shot:")
print(response.output_text)

### Few-Shot for Structured Output

In [ ]:
few_shot_structured = """Extract entities from text and return as JSON:

Text: "Apple announced the iPhone 15 at their Cupertino headquarters."
Output: {"company": "Apple", "product": "iPhone 15", "location": "Cupertino"}

Text: "Microsoft's CEO Satya Nadella spoke at the conference in Seattle."
Output: {"company": "Microsoft", "person": "Satya Nadella", "location": "Seattle"}

Text: "Tesla is building a new factory in Austin, Texas for the Cybertruck."
Output:
"""

response = client.responses.create(
    model="gpt-5-mini",
    input=few_shot_structured,
    temperature=0
)

print(response.output_text)

### 🎯 Exercise 3: Few-Shot for Code Generation

**Task:** Use few-shot learning to generate SQL queries from natural language.

Provide 2-3 examples, then test with a new query.

In [ ]:
# YOUR CODE HERE

sql_prompt = """Convert natural language to SQL:

Database schema:
- users (id, name, email, created_at)
- orders (id, user_id, total, order_date)

TODO: Add 2-3 examples here

Question: "Find all users who made an order in the last 30 days"
SQL:
"""

# Test it
# response = client.responses.create(...)

---

## Part 5: Delimiters & Structure

**Delimiters** clearly separate different parts of your prompt, reducing ambiguity.

**Common delimiters:**
- Triple quotes: `"""`
- Triple backticks: ` ``` `
- XML tags: `<input>`, `<context>`
- Markdown headers: `###`
- Horizontal rules: `---`

### Without Delimiters (Ambiguous)

In [ ]:
no_delimiters = """Summarize this text: The product launch was a success. 
Sales exceeded expectations by 40%. Customer feedback was overwhelmingly positive.
Focus on the key metrics."""

response = client.responses.create(
    model="gpt-5-mini",
    input=no_delimiters
)

print("Without delimiters:")
print(response.output_text)
print("\n" + "="*80 + "\n")

### With Delimiters (Clear)

In [ ]:
with_delimiters = """Summarize the text below, focusing on key metrics:

###
The product launch was a success. Sales exceeded expectations by 40%. 
Customer feedback was overwhelmingly positive.
###
"""

response = client.responses.create(
    model="gpt-5-mini",
    input=with_delimiters
)

print("With delimiters:")
print(response.output_text)

### XML-Style Delimiters for Complex Prompts

In [ ]:
xml_structure = """<task>
Extract the key information and format as JSON.
</task>

<context>
This is a customer support ticket from our CRM system.
</context>

<input>
Customer: Jane Smith
Email: jane@example.com
Issue: Cannot reset password
Priority: High
Timestamp: 2025-10-06 14:30:00
</input>

<output_format>
{
  "customer": "...",
  "email": "...",
  "issue": "...",
  "priority": "...",
  "timestamp": "..."
}
</output_format>
"""

response = client.responses.create(
    model="gpt-5-mini",
    input=xml_structure,
    temperature=0
)

print(response.output_text)

### 🎯 Exercise 5: Using Delimiters

**Task:** Rewrite this ambiguous prompt using clear delimiters:

> "Translate this to Swedish: Hello, how are you? Then explain the translation. Also check if the grammar is correct."

Use XML tags or other delimiters to clearly separate the text to translate, the translation task, and the analysis task.

In [ ]:
# YOUR CODE HERE

structured_translation = """TODO: Rewrite with clear delimiters"""

# response = client.responses.create(
#     model="gpt-5-mini",
#     input=structured_translation
# )

---

## Part 6: Output Formatting

**Structured outputs** ensure consistency and make responses easier to parse programmatically.

**Common formats:**
- JSON
- Markdown tables
- CSV
- Custom templates

### Example: Research Report Generation

In [ ]:
# Step 1: Generate outline
topic = "The impact of AI on software development"

outline_prompt = f"""Create a detailed outline for a report on: {topic}

Include 3-4 main sections with bullet points for each section.
Format as markdown.
"""

response_1 = client.responses.create(
    model="gpt-5-mini",
    input=outline_prompt
)

outline = response_1.output_text
print("Step 1: Generated Outline")
print(outline)
print("\n" + "="*80 + "\n")

In [ ]:
# Step 2: Expand first section
expand_prompt = f"""Based on this outline:

{outline}

Write a detailed 2-paragraph explanation of the FIRST section only.
Use professional tone and include specific examples.
"""

response_2 = client.responses.create(
    model="gpt-5-mini",
    input=expand_prompt,
    previous_response_id=response_1.id  # Chain responses
)

section_1 = response_2.output_text
print("Step 2: Expanded First Section")
print(section_1)
print("\n" + "="*80 + "\n")

In [ ]:
# Step 3: Generate conclusion
conclusion_prompt = f"""Based on this report outline and first section:

OUTLINE:
{outline}

FIRST SECTION:
{section_1}

Write a concise 1-paragraph conclusion that ties together the main points.
"""

response_3 = client.responses.create(
    model="gpt-5-mini",
    input=conclusion_prompt
)

conclusion = response_3.output_text
print("Step 3: Generated Conclusion")
print(conclusion)

### Sequential Data Processing Pipeline

In [ ]:
raw_feedback = """The app is okay I guess. Sometimes it crashes when I upload photos. 
The UI is pretty confusing tbh. But the filters are cool! Support team is super helpful.
Would be nice if it was faster. Overall not bad for a free app."""

# Step 1: Extract key points
extract_prompt = f"""Extract key points from this user feedback:

{raw_feedback}

List each point as a bullet with category (Positive/Negative/Neutral).
"""

response_extract = client.responses.create(
    model="gpt-5-mini",
    input=extract_prompt
)

key_points = response_extract.output_text
print("Extracted Key Points:")
print(key_points)
print("\n" + "="*80 + "\n")

# Step 2: Categorize by urgency
categorize_prompt = f"""Based on these feedback points:

{key_points}

Categorize them by urgency (High/Medium/Low) and suggest priority order for addressing.
Format as a table.
"""

response_categorize = client.responses.create(
    model="gpt-5-mini",
    input=categorize_prompt
)

categorized = response_categorize.output_text
print("Categorized by Urgency:")
print(categorized)
print("\n" + "="*80 + "\n")

# Step 3: Generate action items
action_prompt = f"""Based on this prioritized feedback:

{categorized}

Generate 3 concrete action items for the product team.
Format: [Priority] Action - Expected impact
"""

response_actions = client.responses.create(
    model="gpt-5-mini",
    input=action_prompt
)

print("Action Items:")
print(response_actions.output_text)

### 🎯 Exercise 7: Prompt Chaining

**Task:** Build a 3-step prompt chain for code review:

1. **Identify issues** - Find bugs, code smells, and improvements
2. **Prioritize** - Rank issues by severity
3. **Generate fixes** - Provide code snippets to fix top 3 issues

**Test with this code:**
```python
def calculate_average(numbers):
    total = 0
    for i in range(len(numbers)):
        total = total + numbers[i]
    return total / len(numbers)
```

In [ ]:
# YOUR CODE HERE

code_to_review = """def calculate_average(numbers):
    total = 0
    for i in range(len(numbers)):
        total = total + numbers[i]
    return total / len(numbers)
"""

# Step 1: Identify issues
# TODO: Create prompt

# Step 2: Prioritize issues
# TODO: Create prompt using output from step 1

# Step 3: Generate fixes
# TODO: Create prompt using output from step 2

---

## Summary

In this notebook, you learned:

✅ **Chain-of-Thought (CoT)**: Step-by-step reasoning for complex problems  
✅ **ReAct Pattern**: Combining reasoning with tool use  
✅ **Few-Shot Learning**: Guiding behavior with examples  
✅ **Delimiters & Structure**: Organizing prompts for clarity  
✅ **Prompt Chaining**: Breaking complex tasks into sequential steps  

**Key Takeaways:**
- CoT improves reasoning quality, especially for math and logic
- ReAct is the foundation of agent frameworks (you'll use this in LangGraph!)
- Examples are powerful - few-shot > zero-shot for most tasks
- Clear structure and delimiters prevent ambiguity
- Chaining allows complex workflows with better control

**Next Steps:**
- Notebook 1.3: Structured Outputs (get reliable, validated data)
- Notebook 1.5: Context Management (handle long conversations)
- Apply these patterns in LangGraph agents

**Resources:**
- [Chain-of-Thought Prompting](https://arxiv.org/abs/2201.11903)
- [ReAct Paper](https://arxiv.org/abs/2210.03629)
- [OpenAI Prompt Engineering Guide](https://platform.openai.com/docs/guides/prompt-engineering)